In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000_v2.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=4)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [23]:

#8월대상한정
#fin_result.groupby(['target_m']).count()
target_month = '201907'
fin_result = fin_result[fin_result['target_m'] == target_month]
fin_result = fin_result.reset_index()
#target_fin_result[['target_m','shcode','shname','target_day']].to_csv("201907.csv",encoding = "ms949")

In [11]:
target_fin_result.head()

,index,interval_end,shcode,target_day,target_m,shname
0,0,20190826,000140,20190729,201907,하이트진로홀딩스
1,7,20190813,001820,20190716,201907,삼화콘덴서
2,8,20190822,002070,20190725,201907,남영비비안
3,9,20190823,002070,20190726,201907,남영비비안
4,32,20190801,004830,20190704,201907,덕성


In [ ]:
'''
#1. 이평 정배열 확인한다.

# 대세 판단1 : 월봉 20이평이 증가 중 12개?
# 대세 판단2 : 월봉 5이평이 증가 중
# 대세 판단3 : 월봉 ma5 - ma20  크눈중

#대세 판단 비고 : 외인 비율 확인하기
# 월봉,주봉 이평정배열(이평 5가 20보다 위인 any) or 
# 눌림목 발생하고 ,  거래량 똥 망하고,
# range 사이의 어디구간 30%~50%(구분지어서볼것), 
'''
'''
#2. 시점기준 관련된 주체 확인하여 쿼리로 작성한다. 최근관련된정보 한해서 확인(7,8 등)

'''

In [ ]:
'''
# 눌림목 발생하고 
sqeezed_bool = abs(df['close'] /df['open'] - 1) <= 0.03
#거래량 3/100 망하고,
volume_bool = df['volume'] / df['volume'].iloc[0] <= 0.03
# range 사이의 범위
spot_range_value = (df['close'] - df['open'].iloc[0]) / (df['close'].iloc[0] - df['open'].iloc[0])
# inrange 여부
inrange_bool = (df['close'] >= df['open'].iloc[0])&(df['close'] <= df['close'].iloc[0])

test_list = pd.DataFrame(dict(date = np.array(df)[:,0][1:],
                              close_ratio = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],
                              close = df['close'][1:],
                              spot_range = spot_range_value[1:],
                              check = sqeezed_bool[1:] * volume_bool[1:] * inrange_bool[1:],
                              squeezed = sqeezed_bool[1:],
                              volume_default = volume_bool[1:],
                              inrange = inrange_bool[1:]
                             ))


'''
'''
# daily open <= open
open_touch_list = df['date'].iloc[1:][df['open'] <= open_trigger]
# daily close >= close
close_touch_list = df['date'].iloc[1:][df['close'] >= close_trigger]

close_touch_list
'''

In [3]:
def non_decreasing(L):
    data = [x<=y for x, y in zip(L, L[1:])]
    return data

In [ ]:
amount = [0.15, 0.15, 0.2, 0.2, 0.3]
# 대세 판단1 : 월봉 5이평이 감소하지않음
ma5 = non_decreasing(df['close_ma5'])[-5:]
ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)
# 대세 판단2 : 월봉 20이평이 감소하지않음
ma20 = non_decreasing(df['close_ma20'])[-5:]
ma20_score = sum(x * y for x, y in zip(ma20, amount)) / sum(amount)
# 대세 판단3 : 월봉 ma5 - ma20  크눈중
ma5_20 = non_decreasing(df['close_ma5']-df['close_ma20'])[-5:]
ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)

In [59]:
fin_result['target_day'].iloc[i][:6]+

'201906'

In [65]:
fin_result['target_day'].iloc[i]

'20190614'

In [64]:
str(int(fin_result['target_day'].iloc[i][:6]) + 1) + '01'

'20190701'

In [25]:
fin_result.shape

(57, 6)

In [28]:
fin_result.head()

,index,interval_end,shcode,target_day,target_m,shname
0,0,20190826,000140,20190729,201907,하이트진로홀딩스
1,7,20190813,001820,20190716,201907,삼화콘덴서
2,8,20190822,002070,20190725,201907,남영비비안
3,9,20190823,002070,20190726,201907,남영비비안
4,32,20190801,004830,20190704,201907,덕성


In [ ]:
변수 기간 등

In [7]:
fin_result.iloc[384:]

,interval_end,shcode,target_day,target_m,shname
384,20190227,198440,20190130,201901,고려시멘트
385,20190705,198440,20190607,201906,고려시멘트
386,20190321,203650,20190221,201902,드림시큐리티
387,20190716,203650,20190618,201906,드림시큐리티
388,20190508,205100,20190410,201904,엑셈
389,20190208,206560,20190111,201901,덱스터
390,20190211,206560,20190114,201901,덱스터
391,20190514,206560,20190416,201904,덱스터
392,20190726,207760,20190628,201906,미스터블루
393,20190801,207760,20190704,201907,미스터블루


In [22]:
rrr = pd.DataFrame([],columns = ['date',  'close_ratio','close', 
                                 'min_target', 'max_target', 'spot_range',
                                'check', 'inrange', 'squeezed', 'volume_default',
                                'shcode','shname', 'target_m'])

for i in range(len(fin_result['shcode'])):
    # 월봉 대세판단
    df_monthly = query.query_ohlcv_month(shcode = fin_result['shcode'].iloc[i], 
                                      to = str(int(fin_result['target_day'].iloc[i][:6]) + 1) + '01')
    
    #except1: data없는경우
    if df_monthly.shape[0] ==0:
        continue
    
    # ma preprocesser
    df_monthly = preprocesser.sma_mothly(df_monthly)
    
    ma5_20 = non_decreasing(df_monthly['close_ma5']-df_monthly['close_ma20'])[-5:]
               
    amount = [0.15, 0.15, 0.2, 0.2, 0.3]
    ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)
    
    #except2: 이평선제외하자
    if ma5_20_score == 0:
        continue

    #일봉 
    df = query.query_ohlcv(shcode = fin_result['shcode'].iloc[i], 
                          fr = fin_result['target_day'].iloc[i], 
                          to = fin_result['interval_end'].iloc[i])
    
    range_open = df['open'].iloc[0]
    range_close = df['close'].iloc[0]    
               
    # 눌림목 발생하고     
    sqeezed_bool = abs(df['close'] /df['open'] - 1) <= 0.03
    #거래량 3/100 망하고,
    volume_bool = df['volume'] / df['volume'].iloc[0] <= 0.03
    # range 사이의 범위
    spot_range_value = (df['close'] - range_open) / (range_close - range_open)
    # inrange 여부
    inrange_bool = (df['close'] >= range_open)&(df['close'] <= range_close)

    test_list = pd.DataFrame(dict(date = np.array(df)[:,0][1:],
                                  close_ratio = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],
                                  min_target = range_open,
                                  max_target = range_close,
                                  close = df['close'][1:],
                                  spot_range = spot_range_value[1:],
                                  check = sqeezed_bool[1:] * volume_bool[1:] * inrange_bool[1:],
                                  inrange = inrange_bool[1:],
                                  squeezed = sqeezed_bool[1:],
                                  volume_default = volume_bool[1:]
                                 ))
    test_list['shcode'] = fin_result['shcode'].iloc[i]
    test_list['shname'] = fin_result['shname'].iloc[i]
    test_list['target_m'] = fin_result['target_m'].iloc[i]
    rrr = rrr.append(test_list)
    print(i , fin_result['shcode'].iloc[i] , "완료")

0 000140 완료
1 000890 완료
2 001140 완료
3 001340 완료
6 001745 완료
8 002070 완료
9 002070 완료
10 002140 완료
11 002140 완료
12 002140 완료
13 002150 완료
16 002320 완료
17 002360 완료
18 002450 완료
19 002700 완료
20 002720 완료
21 002780 완료
22 002780 완료
23 002995 완료
24 003010 완료
25 003495 완료
27 003850 완료
28 004060 완료
29 004270 완료
30 004720 완료
31 004770 완료
32 004830 완료
34 004980 완료
35 005110 완료
36 005110 완료
37 005320 완료
38 005360 완료
39 005390 완료
42 005740 완료
43 006090 완료
44 006890 완료
45 006980 완료
46 006980 완료
47 006980 완료
48 007110 완료
49 007110 완료
50 007210 완료
52 007540 완료
53 007540 완료
54 008040 완료
55 008350 완료
57 008700 완료
58 008730 완료
59 008775 완료
60 009290 완료
61 009320 완료
62 009320 완료
64 009410 완료
65 009680 완료
66 009680 완료
67 010100 완료
68 010100 완료
69 010420 완료
70 010660 완료
71 011150 완료
72 011150 완료
73 011150 완료
74 011280 완료
75 011280 완료
76 011390 완료
77 012160 완료
78 012450 완료
79 012610 완료
80 012690 완료
81 012800 완료
82 012800 완료
83 013700 완료
84 014130 완료
85 014580 완료
86 014580 완료
87 014580 완료
88 014580 완료
89 014

In [25]:
rrr[['shcode', 'shname', 'target_m', 'date','close_ratio', 'min_target','max_target'
    ,'close','spot_range','check','inrange','squeezed','volume_default']].to_csv("3.csv",encoding = 'ms949')

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')